<span style="display:block;text-align:center;margin-right:105px"></span>

# Blockchain based ROSCA - version 1
---

* [System Requirements (Part 1)](#System-Requirements)
  * [Model Introduction](#Model-Introduction)
  * [cadCad Overview](#cadCad-Overview)
  * [Requirements Analysis](#Requirements-Analysis)
  * [Visual System Mapping: Entity Relationship Diagram](#Visual-System-Mapping:-Entity-Relationship-Diagram)
  * [Visual System Mapping: Stock & Flow Diagram](#Visual-System-Mapping:-Stock-&-Flow-Diagram)
  * [Mathematical Specification](#Mathematical-Specification)
* [System Design (Part 2)](#System-Design)
  * [Differential Specification](#Differential-Specification)
  * [cadCAD Standard Notebook Layout](#cadCAD-Standard-Notebook-Layout)
    0. [Dependencies](#0.-Dependencies)
    1. [State Variables](#1.-State-Variables)
    2. [System Parameters](#2.-System-Parameters)
    3. [Policy Functions](#3.-Policy-Functions)
    4. [State Update Functions](#4.-State-Update-Functions)
    5. [Partial State Update Blocks](#5.-Partial-State-Update-Blocks)
    6. [Configuration](#6.-Configuration)
    7. [Execution](#7.-Execution)
    8. [Simulation Output Preparation](#8.-Simulation-Output-Preparation)
    9. [Simulation Analysis](#9.-Simulation-Analysis)
* [System Validation (Part 3)](#System-Validation)
    * [Policy Functions](#Policy-Functions)
    * [Model Improvements](#Model-Improvements)
      * [Differential Specification Updates](#Differential-Specification-Updates)
      * [Mathematical Specification Updates](#Mathematical-Specification-Updates)
    * [Model Limitations](#Model-Limitations)
---

# System Requirements

## Model Introduction
> ROSCAs are informal saving and credit institutions, whereby members pool together regular savings with are then dispersed to a member, either by random or based on the greatest need. This continues until every member has received the pot. 

ROSCAs are traditionally operated in small group within communities. I have found ROSCAs stand to be improved using blockchain technology - my review has demonstrated that blockchain technology certainly has the potential to create a more democratic, transparent truly peer-to-peer system. This has been proven by groups such as <a href="https://tlc.wetrust.io/">WeTRUST</a> who are attempting to: `"Create a blockchain-based Lending Circle dApp promoting financial inclusion and lay the groundwork for new financial tools for unbanked and underserved people throughout the world."` 

Having explored the profound social and economic implications of decentralized ledger technologies on ROSCAs, there is an implied pressing need for sustainable system design, which brings us to cadCad (complex adaptive dynamics Computer-aided design) which helps in designing, optimising, validating and understanding trivially non-complex, dynamical systems.

## cadCad Overview
cadCAD: Python library, assisting in the process of designing, testing and validating complex/economic systems through simulation. Allows you to ask powerful 'what-if' questions about your system, using simulation features such as A/B tests, parameter sweeps and Monte-Carlo analysis. 

These 'what-if' questions can concern system internal factors (eg. agent behaviours, system parameters) or system external factors (eg. market conditions, competitive activity). cadCad can model systems with various levels of abstractions from micro-level agent modelling to macro-level system dynamics.


## Requirements Analysis
Our goal for version 1 is to get an initial ROSCA circle running in a cadCad simulation environment. 
In this notebook, we want to: 
1. Present the dynamics of a single 'trusted' ROSCA circle on a peer-to-peer centralised network where agents share a common view of the chain 
2. Define simple strategies for agents to: intiate intial ROSCA circle, be set a contribution amount set by initiator, be allocated the pot in a random order
3. Understand the basic space of decision making of several players in the protocol

With this information, hopefully having demonstrated we can, in fact, reach our goal of `a fair, democratic, decentralised model solution of ROSCA`, we can build off of this version to implement:
1. Neccessary mechanisms eg: voting, borrowing 
2. Design incentives
3. Implement two types of circles (trusted & non-trusted)
4. Understand how agent behaviours map to the incentives (rewards, penaties, general outcomes)
5. Produce rich agent-based models

### Questions
1. What sort of mechanisms/incentives would we need to ensure agents parcipate in a way that fufils the systems goals
2. What are the penalties to deincentivise dishonest behaviour
3. What information is available to agents when their decisions are made


## Visual System Mapping: Entity Relationship Diagram

![Entity Relationship diagram](./images/entity.png)

At its simplest, the system allows for two core actions: `a participant contributing to the pool`, and `a participant receiving from the pool`. This takes into account a participants standing within the circle.

## Visual System Mapping: Stock & Flow Diagram

![Stock & Flow Diagram](./images/stockflow.png)

Represents and encodes the mathematical structure of a model. We identify a single stock, money pool, which gets its input from the source c, contributions from members and has a sink a, allocating to a participant in a given month.

## Mathematical Specification

![Math Spec](./images/spec.png)

The above differential equation gives a more formal mathematical representation of the 'money pool' physical quantity. The money in the pool at a given timestep, t is equal to the amount in the previous timestep, add c, the total sum of monthly agent contribution amount, minus a, the the amount allocated to agent (their monthly contribution * duration) 

---

# System Design

## Differential Specification

![Differential Spec](./images/diff.png)

In the cadCAD simulation methodology, we operate on four layers: Policies, Mechanisms, States, and Metrics. 
* Policies determine the inputs into the system dynamics, and can come from user input, observations from the exogenous environment, or algorithms. 
* Mechanisms are functions that take the policy decisions and update the States to reflect the policy level changes.
* States are variables that represent the system quantities at the given point in time 
* Metrics are computed from state variables to assess the health of the system. Metrics can often be thought of as KPIs, or Key Performance Indicators.

## cadCAD Standard Notebook Layout
<center>
<img src="./images/cadcad-flow.png"
     alt="cadCAD flow"
     style="width: 25%;" />
</center>

# 0. Dependencies

In [1]:
# Standard libraries: https://docs.python.org/3/library/
import math
import secrets

# Analysis and plotting modules
import pandas as pd

# Logic
import secrets
import base_blockchain
from agents import (AgentData, update_contribution_amount)
from circle import (initialise_circle_state, update_circle_state, get_total_active_balance)
from chain import(create_block)
from utils import (pounds_to_eth, eth_to_pounds)

In [2]:
# cadCAD configuration modules
from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment

# cadCAD simulation engine modules
from cadCAD.engine import ExecutionMode, ExecutionContext
from cadCAD.engine import Executor

blockchain = base_blockchain.BlockChain()

# 1. State Variables

> A state variable is one of the set of variables that are used to describe the mathematical "state" of a dynamical system. ([Wikipedia](https://en.wikipedia.org/wiki/State_variable))

### Genesis state and genesis block
Dynamical systems are usually rather compact to define. We need at the very least two things:
1. Some initial state.
2. The dynamics, or how state evolves over time.

In this part i'll focus on the intial state. 
We'll create a genesis state, with 10 participants who each are set to pay 0.21 ETH (£200) into the money pool for a set term of 10 months, using a randomised turn system.

In [3]:
# Create an array of `AgentData` objects
def get_initial_deposits(n):
    agent = [AgentData(secrets.token_bytes(48), pounds_to_eth(300), i, 0, False, 0)
             for i in range(n)]
    return agent

In [4]:
# intialise a rosca circle 
circle_ = initialise_circle_state("Hudas saving circle :)", get_initial_deposits(10))

NameError: name 'circle' is not defined

In [ ]:
# create initial block for the groups blockchain
genesis_state = create_block(circle_)


![Code Layout](./images/layout.png)
Now we have create our rosca circle and have processed a genesis block, with the code workflow above for clarity. Finally, we set the genesis state as the initial condition to our cadCAD execution

In [ ]:
initial_conditions = {
    'pool_state': genesis_state
}

## **Time** as a system state
* 1 **timestep** == 1 month

# 2. System Parameters
> System parameterization is the process of choosing variables that impact the behaviour of the model. These parameters allow us to perform simulation techniques like parameter sweeps, Monte Carlo simulations, A/B tests, and see how the system behaves under a different model parameter set.

In [ ]:
system_params = {
    'duration': [10], # duration of rosca circle
}
system_params

# 3. Policy Functions

> A Policy Function computes one or more signals to be passed to State Update Functions. They describe the logic  and behaviour of a system component  or mechanism.

# 4. State Update Functions
> We create State Update Functions to design the way our model state changes over time. These will usually represent the system differential specification.

```python
def state_update_function(params, substep, state_history, previous_state, policy_input):
    variable_value = 0
    return 'variable_name', variable_value
```

* `params` is a Python dictionary containing the **system parameters** <!-- for consistency with the previous definition -->
* `substep` is an integer value representing a step within a single `timestep`
* `state_history` is a Python list of all previous states
* `previous_state` is a Python dictionary that defines what the state of the system was at the **previous timestep** or **substep**
* `policy_input` is a Python dictionary of signals or actions from **policy functions**

### State evolution with block proposals
We can now move on to defining the dynamics. Simple dynamical systems follow some rules to update their state, e.g., the orbits of planets are given by laws derived from attraction. More complex systems are controlled: decisions made by agents in the system influence the state evolution.

In our setting, these agents are ROSCA participants. State evolution is governed by the rules set out by the system, while participants get to decide on putting in a transaction or defaulting. The resulting state is a combination of agent decisions and state updates. How participants make these decisions is embodied by policy functions: given a state, make a decision.

At the start of the process, the `genesis_state` defined above is `M(t-1)`, to update the state of the moneypool to `M(t)` we need to calculate `c - the amount collected from agents` and `α - the amount which is allocated`. We break this down into 3 steps:

#### 1. Initiate monthly collections from each agent in the rosca circle

In [ ]:
# initate monthly contributions
def intitiate_monthly_deposits(genesis_state):
    month_sum = 0.0
    block_agent_data = genesis_state.data[0]['data']

    for x in range(0, len(block_agent_data)):
        agent_contribution = float(block_agent_data[x].amount)
        month_sum += eth_to_pounds(agent_contribution) # c - from differential eq
        block_agent_data[x].contributed_amt += agent_contribution # update agent contributed amount

    print("Deposited amount: GBP" + str(month_sum))
    return month_sum    # todo: validate sum at later stage

#### 2. Allocate pool to agent according to standing within the circle 

In [ ]:
# Allocate to agent according to standing
def allocate_pool(genesis_state, timestep, duration):
    block_agent_data = genesis_state.data[0]['data']
    pool_amount = intitiate_monthly_deposits(genesis_state)

    for x in range(0, len(block_agent_data)):
        if(block_agent_data[x].standing == timestep and block_agent_data[x].taken == False):
            allocation_amt = (duration*block_agent_data[x].amount) # a - from differential eq
            block_agent_data[x].balance += allocation_amt
            block_agent_data[x].taken = True
            print("Allocating agent: " + str(block_agent_data[x].pubkey) + " amount: ETH" + str(allocation_amt))
            pool_amount -= eth_to_pounds(allocation_amt)
    return pool_amount

#### 3. Update state of circle and proccess and add block

In [ ]:
def new_pool_state(genesis_state, timestep, duration):    
    updateCircle_ = update_circle_state(genesis_state.data[0]['name'], timestep+1, allocate_pool(genesis_state, timestep, duration), genesis_state.data[0]['data'])
    new_block = create_block(updateCircle_) # create block
    print("Pool amount: GBP" + str(get_total_active_balance(updateCircle_)))
    
    return updateCircle_

In [ ]:
# Relevant state variables
current_pool_state = initial_conditions['pool_state']

# Relevant parameters
duration = system_params['duration'][0] #  from "alpha" in our differential equation

new_state = new_pool_state(current_pool_state, 1, duration)

In [ ]:
def s_pool(params, substep, state_history, previous_state, policy_input):
    """
    Update the pool state according to the differential equation (1):
    current_pool_state + collected_amount - allocation_amount
    """
    pool = previous_state['pool_state']
    duration = params['duration']
    
    return 'pool_state', new_pool_state(pool, substep, duration)

In [ ]:
next_state = {
    'pool_state': new_state
}
next_state

# 5. Partial State Update Blocks
> A series of Partial State Update Blocks is a structure for composing State Update Functions and Policy Functions in series or parallel, as a representation of the system model. 

<center>
<img src="./images/basic-psub.png"
     alt="Policy functions"
     style="width: 60%;" />
</center>

In [ ]:
partial_state_update_blocks = [
    {
        'policies': {}, # Ignore for now
        # State variables
        'variables': {
            # Updated in parallel
            'pool_state': s_pool,
        }
    }
]

# 6. Configuration
> The configuration stage is about tying all the previous model components together and choosing how the simulation should run.

![Flow](./images/cadflow2.png)

Configuration parameters:
* `'N': 1` - the number of times we'll run the simulation (you'll see them called "Monte Carlo runs" later in the course, when we look at tools to analyze system models)
* `'T': range(400)` - the number of timesteps the simulation will run for
* `'M': system_params` - the parameters of the system

In [ ]:
sim_config = config_sim({
    "N": 1,
    "T": range(10),
    "M": system_params
})

In [ ]:
from cadCAD import configs
del configs[:] # Clear any prior configs

In [ ]:
experiment = Experiment()
experiment.append_configs(
    initial_state = initial_conditions,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config
)
configs[-1].__dict__

# 7. Execution

> The Execution Engine takes a model and configuration, and computes the simulation output.

## Configuring the cadCAD simulation execution

In [ ]:
exec_context = ExecutionContext()

In [ ]:
simulation = Executor(exec_context=exec_context, configs=configs)

## We can now simulate our ecosystem model!

In [5]:
raw_result, tensor_field, sessions = simulation.execute()

NameError: name 'config' is not defined

# 8. Simulation Output Preparation
> The simulation results are returned as a list of Python dictionaries, which we then convert to a Pandas dataframe. At this stage of the process you'll manipulate and analyze your results to answer questions about your model.

In [ ]:
simulation_result = pd.DataFrame(raw_result)

In [ ]:
raw_result[:5]

In [ ]:
simulation_result.head()

# 9. Simulation Analysis